In [8]:
from sklearn.preprocessing import OneHotEncoder    
from sklearn.compose import ColumnTransformer, StandardScaler
import pandas as pd

In [51]:
train = pd.read_csv("./store-sales-time-series-forecasting/train.csv",parse_dates=['date'], index_col = ['id'])

C:\Users\yongjin\Anaconda3\envs\edu_machine_learning\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [52]:
train.columns

Index(['date', 'store_nbr', 'family', 'sales', 'onpromotion'], dtype='object')

In [53]:
categorical_features = ["family", "store_nbr"]

In [54]:
train_categorical = train[categorical_features]

In [55]:
train_non_categorical = train.drop(categorical_features, axis =1)

In [56]:
enc = OneHotEncoder(handle_unknown= 'ignore')

In [57]:
train_categorical = enc.fit_transform(train_categorical).toarray()

In [58]:
pd.concat([train_non_categorical, pd.DataFrame(train_categorical)], axis = 1)

,date,sales,onpromotion,0,1,2,3,4,5,6,...,77,78,79,80,81,82,83,84,85,86
0,2013-01-01,0.000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-01-01,0.000,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-01-01,0.000,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-01-01,0.000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-01-01,0.000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,2017-08-15,438.133,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3000884,2017-08-15,154.553,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3000885,2017-08-15,2419.729,148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3000886,2017-08-15,121.000,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
